---
toc: false
comments: true
layout: post
title: Team Teach - Full Stack and User Profile
description: CRUD backend and frontend, Authentication
type: hacks
courses: { compsci: {week: 19} }
---

# CRUD

## What is CRUD?

CRUD stands for:

C-reate: Involves the creation of addition of data to a database. In essence, it "create"s entries in databases.

R-ead: Involves the retrieval, querying, or "read"ing of existing data from a database.

U-pdate: Involves the modification or "update"ing of existing data in a database. Operation of changing the values of specific fields in a system. 

D-elete: Involves the removal or "delete"ion of existing data from a database. It's the operation of eliminating information from a database.

In the past, each function has had its own link to fetch from, but with CRUD all four functions will be under one link, differentiated by the request type most suited for it.
Create: POST request, as create operation needs to sent data back to the backend, best suited by post request
Read: GET request, as read operation is simple and requires no data to be sent back
Update: PUT request, as update requires the user ID of the user being updated, and this can be modified in the URL which PUT requests allow
Delete: DELETE request, self explanatory
This is simpler and makes fetching less confusing, as everything is done under one link. 

## Authentication
- How users will authenticate to be able to access function that require a token
- POST request is sent to backend
    - payload contains user ID and password
- Backend compares user ID and password, and returns with token if successful
- Note that authentication and authorization are two different things
    - Authentication is identifying who you are, client saying hi to backend
    - Authorization is getting permission to view something, website letting client know that they can view something

#### Frontend

In [ ]:
function login_user(){
    // Set Authenticate endpoint
    const url = 'http://127.0.0.1:8086/api/users/authenticate';

    // Set the body of the request to include login data from the DOM
    const body = {
        uid: document.getElementById("uid").value,
        password: document.getElementById("password").value,
    };

    // Change options according to Authentication requirements
    const authOptions = {
        mode: 'cors', // no-cors, *cors, same-origin
        credentials: 'include', // include, same-origin, omit
        headers: {
            'Content-Type': 'application/json',
        },
        method: 'POST', // Override the method property
        cache: 'no-cache', // Set the cache property
        body: JSON.stringify(body)
    };

    // Fetch JWT
    fetch(url, authOptions)
    .then(response => {
        // handle error response from Web API
        if (!response.ok) {
            const errorMsg = 'Login error: ' + response.status;
            console.log(errorMsg);
            alert("Failed Authentication: Credentials Incorrect")
            return;
        }
        // Success!!!
        // Redirect to the database page
        window.location.href = "/csp-blog//log/2024/01/30/Users.html";
    })
    // catch fetch errors (ie ACCESS to server blocked)
    .catch(err => {
        console.error(err);
    });
}

// Attach login_user to the window object, allowing access to form action
window.login_user = login_user;

#### Backend

In [ ]:
'''
import the necessary packages for different crud operations
'''
import json
from flask import Blueprint, request, jsonify
from flask_restful import Api, Resource # used for REST API building
from datetime import datetime

# check password parameter versus stored/encrypted password
def is_password(self, password):
    """Check against hashed password."""
    result = check_password_hash(self._password, password)
    return result

In [ ]:
def post(self):
            try:
                body = request.get_json()
                if not body:
                    return {
                        "message": "Please provide user details",
                        "data": None,
                        "error": "Bad request"
                    }, 400
                ''' Get Data '''
                uid = body.get('uid')
                if uid is None:
                    return {'message': f'User ID is missing'}, 400
                password = body.get('password')
                
                ''' Find user '''
                user = User.query.filter_by(_uid=uid).first()
                if user is None or not user.is_password(password):
                    return {'message': f"Invalid user id or password"}, 400
                if user:
                    try:
                        token = jwt.encode(
                            {"_uid": user._uid},
                            current_app.config["SECRET_KEY"],
                            algorithm="HS256"
                        )
                        resp = Response("Authentication for %s successful" % (user._uid))
                        resp.set_cookie("jwt", token,
                                max_age=3600,
                                secure=True,
                                httponly=False,
                                path='/',
                                samesite='None'  # This is the key part for cross-site requests

# creates a new blueprint for the user API, using Blueprint from flask library
user_api = Blueprint('user_api', __name__,
                   url_prefix='/api/users')

# API docs https://flask-restful.readthedocs.io/en/latest/api.html
# Flask-RESTful API instance
api = Api(user_api)

# define our class for the user api
# two nested classes, one for CRUD operations and the other for security/login
class UserAPI:   
         
    class _CRUD(Resource): # User API operation for Create, Read.  THe Update, Delete methods need to be implemeented
        def post(self): # Create method
            ''' Read data for json body '''
            body = request.get_json()
            
            ''' Avoid garbage in, error checking '''
            # validate name
            name = body.get('name')
            if name is None or len(name) < 2:
                return {'message': f'Name is missing, or is less than 2 characters'}, 400
            # validate uid
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 400
            # look for password
            password = body.get('password')
            
            '''below class information is specific for users on the nighthawkcoders website'''

            active_classes = body.get('active_classes')
            if active_classes is None or len(active_classes) == 0:
                return {'message': 'Active class is missing'}, 400

            archived_classes = body.get('archived_classes')

            ''' #1: Key code block, setup USER OBJECT '''
            uo = User(name=name, 
                      uid=uid,
                      active_classes=active_classes,
                      archived_classes=archived_classes)
            
            ''' Additional garbage error checking '''
            # set password if provided
            if password is not None:
                uo.set_password(password)
            # set server request status, specific to cs classes
            uo.server_needed = body.get('server_needed')
            
            ''' #2: Key Code block to add user to database '''
            # create user in database
            user = uo.create()
            # success returns json of user
            if user:
                return jsonify(user.read())
            # failure returns error
            return {'message': f'Processed {name}, either a format error or User ID {uid} is duplicate'}, 400

In [ ]:
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

## Read
#### Frontend
This example read code sends a GET request to the backend. Note that read doesn't necessarily mean returning the entire database, but simply reading already existing information from the database. Note that the javaScript below creates a table with the retrieved information but other things can be done with it.

In [ ]:
// uri variable and options object are obtained from config.js
  import { uri, options } from '/teacher_portfolio/assets/js/api/config.js';

  // Set Users endpoint (list of users)
  const url = uri + '/api/users/';

  // prepare HTML result container for new output
  const resultContainer = document.getElementById("result");

            ''' Update fields '''
            # grab name, update if exists
            name = body.get('name')
            if name:
                user.name = name
            # update uid    
            uid = body.get('uid')
            if uid:  
                user.uid = uid
            
            # server_needed and classes is specific to nighthawkcoders again    
            user.server_needed = body.get('server_needed')

            user.active_classes = body.get('active_classes')
            user.archived_classes = body.get('archived_classes')

            ''' Commit changes to the database '''
            user.update()
            return jsonify(user.read()) # return new changes to user
        
        def delete(self, id):  # Delete method
            ''' Find user by ID '''
            user = User.query.get(id)
            if user is None:
                return {'message': f'User with ID {id} not found'}, 404

            ''' Delete user '''
            user.delete()
            return {'message': f'User with ID {id} has been deleted'}

    # nested class, for security and login page
    class _Security(Resource):

        def post(self):
            '''Authenticate User'''
            # Read data for json body
            body = request.get_json()
            
            # Get Data
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 400
            password = body.get('password')
            
            # Find user
            user = User.query.filter_by(_uid=uid).first()
            if user is None or not user.is_password(password):
                return {'message': f"Invalid user id or password"}, 400
            
            ''' authenticated user '''
            return jsonify(user.read())

# note that there was a lot of error-handling in the event of edge cases
# such as in the case of invalid uids, usernames, etc.
            
    # building RESTapi endpoint
    # we call the classes from inside UserAPI
    api.add_resource(_CRUD, '/', '/<int:id>') 
    api.add_resource(_Security, '/authenticate')
    